In [1]:
import gym
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #### REMOVE THIS LINE WHEN CUDA CONFIG IS FIXED
import tensorflow as tf
import numpy as np
import sys
import time
import gym_snake
import json
import importlib
import random
from PIL import Image

sys.path.insert(0, '..')
from algos.PPO import PPO_agent

# %load_ext line_profiler
%matplotlib notebook

In [2]:
# tetris = importlib.import_module('pytris-effect.src.gameui')

In [3]:
run_name = 'snake'
action = 'train'

In [4]:
cfg_fp = os.path.join('..', 'configs', run_name + '.json')
with open(cfg_fp, 'r') as f:
    config = json.load(f)
ckpt_folder = os.path.join('..', 'checkpoints')

In [5]:
env_name = config['env']
if run_name == 'tetris':
    env = tetris.GameUI(graphic_mode=False, its_per_sec=2, sec_per_tick=0.5)
else:
    env = gym.make(env_name).env if 'use_raw_env' in config else gym.make(env_name)

In [6]:
env.reset().shape

(150, 150, 3)

In [7]:
def show_img(arr):
    scaling = 30
    data = np.zeros((scaling*arr.shape[0], scaling*arr.shape[1], 3), dtype=np.uint8)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            for k in range(data.shape[2]):
                data[i,j,k] = arr[i//scaling,j//scaling,k]
    img = Image.fromarray(data, 'RGB')
    # img.save('my.png')
    img.show()

In [8]:
if action == 'evaluate':
    %lprun -f env.drawMatrix env.drawMatrix()

In [9]:
#show_img(env.reset())

In [10]:
if False:
    action = 1
    obs, reward, dn, info = env.step(action)
    show_img(obs)
    print(reward, dn, info)

In [11]:
def do_step():
    _, _, dn, _ = env.step(random.choice(range(7)))
    if dn:
        env.reset()

In [12]:
#%timeit env.reset()

In [13]:
#%timeit do_step()

In [14]:
#%lprun -f env.get_obs do_step()

In [15]:
if env_name == "CartPole-v0":
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(4,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    value = tf.keras.Sequential([
        tf.keras.Input(shape=(4,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation=None)
    ])
elif env_name == "MountainCar-v0":
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(2,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
elif env_name == "Acrobot-v1":
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(6,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(48, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    value = tf.keras.Sequential([
        tf.keras.Input(shape=(6,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(48, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation=None)
    ])
elif env_name == "gym_snake:snake-v0":
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(15, 15, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        #tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        #tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    value = tf.keras.Sequential([
        tf.keras.Input(shape=(15, 15, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        #tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        #tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation=None)
    ])
elif env_name == "tetris":  # the final raid boss
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(20,10,3)),
        tf.keras.layers.Conv2D(32, (2, 2), activation='elu', padding='same'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='valid'), # new addition
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(256, activation='elu'),
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dense(7, activation='softmax') # NO-OP is an action
    ])
    value = tf.keras.Sequential([
        tf.keras.Input(shape=(20,10,3)),
        tf.keras.layers.Conv2D(32, (2, 2), activation='elu', padding='same'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='valid'), # new addition
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(256, activation='elu'),
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dense(1, activation=None)
    ])

In [16]:
agent = PPO_agent(
    model,
    value,
    env=env,
    learning_rate=config['learning_rate'],
    minibatch_size=config['minibatch_size'],
    epsilon=0.1,
    gamma=0.95,
    env_name=config['env_name'],
    run_name='snake-PPO-16-54-23',
    ckpt_folder=ckpt_folder
)

/home/thebes/Documents/ML/RL/venv/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [17]:
t_max = config['t_max']

In [18]:
agent.load_from_checkpoint()

In [19]:
if action == 'train':
    agent.train(epochs=config['train_epochs'], t_max=t_max, buf_size=3000, min_buf_size=500)

Training epochs:   0%|          | 0/3000 [00:00<?, ?it/s]

[5] Average reward: 0.8125
Saving to checkpoint...
[10] Average reward: 0.95
Saving to checkpoint...
[15] Average reward: 0.90625
Saving to checkpoint...
[20] Average reward: 0.99375
Saving to checkpoint...
[25] Average reward: 0.73125
Saving to checkpoint...
[30] Average reward: 0.575
Saving to checkpoint...
[35] Average reward: 0.475
Saving to checkpoint...
[40] Average reward: 0.46875
Saving to checkpoint...


KeyboardInterrupt: 

In [ ]:
def test_rollout(t_max, env, close=True):
    import sys
    obs = agent.preprocess(env.reset())
    reward = 0
    for i in range(t_max):
        # print(agent.get_policy(obs))
        act = agent.get_action(obs, greedy=True)[0]
        obs, r, dn, info = env.step(agent.action_wrapper(act))
        env.render()
        print(act, file=sys.stderr)
        time.sleep(0.05)
        obs = agent.preprocess(obs)
        reward += r
        if dn:
            break

    print("Total reward: {}".format(reward), file=sys.stderr)
    if close: env.close()

In [ ]:
if action == 'test':
    test_rollout(10000, env, close=False)

In [ ]:
# agent.train(4, t_max=500, min_buf_size=10)

In [ ]:
# %lprun -f agent.train agent.train(1, t_max=500, buf_size=2000, min_buf_size=10)